En este archivo puedes escribir lo que estimes conveniente. Te recomendamos detallar tu solución y todas las suposiciones que estás considerando. Aquí puedes ejecutar las funciones que definiste en los otros archivos de la carpeta src, medir el tiempo, memoria, etc.

# Llamada del archivo para su procesamiento

In [1]:
file_path = r"D:\\Personal\\Prueba_Tecnica\\LATAM\\challenge_DE\\src\\farmers-protest-tweets-2021-2-4.json"

In [ ]:
#pip install py-spy
#pip install pyspy
#pip install memory-profiler
#pip install emoji

In [2]:
import pandas as pd
import numpy as np
import json
from pandas import json_normalize
import cProfile
from typing import List, Tuple
from datetime import datetime
from memory_profiler import memory_usage
import emoji
from collections import Counter

## Validaciones iniciales

In [3]:
#Cara del df 

data = []
# Crear conjunto para la claves únicas 
unique_keys = set()

with open(file_path, 'r') as file:
    for line in file:
        json_data = json.loads(line.strip())
        data.append(json_data)
        unique_keys.update(json_data.keys())

#Crear el dataframe inicial en función de los datos leídos y con columnas nombradas tal como las claves únicas obtenidas
df_1 = pd.DataFrame(data).reindex(columns=unique_keys)

**Duplicados**

La validación de duplicados se realiza sobre la columna ID, dado que esta la que contiene los registros únicos de los twits

In [4]:
df_1['id'].duplicated().sum()

0

**Vacíos**

In [5]:
df_1.isnull().sum()/len(df_1)

conversationId     0.000000
content            0.000000
quotedTweet        0.647074
user               0.000000
media              0.760585
sourceLabel        0.007768
sourceUrl          0.007768
retweetedTweet     1.000000
replyCount         0.000000
url                0.000000
renderedContent    0.000000
retweetCount       0.000000
tcooutlinks        0.000000
mentionedUsers     0.676050
lang               0.000000
quoteCount         0.000000
id                 0.000000
likeCount          0.000000
outlinks           0.000000
source             0.000000
date               0.000000
dtype: float64

Respecto a esto se identifca que la columna retweetedTweet tiene el 100% de datos nulos, en su momento si para algún punto se consideran eliminar las columna con alto % de vacío se procederá, de lo contrario permanecerá.

Por ejemplo, las columans mentioneUsers se podría esperar un alto % de vacíos dado que en todos los comentarios no necesariamente se etiqueta otra cuenta de "X".

## Punto 1 datos temporales

### **Función inicial**

**Consideraciones con las que se creó la función**

1. Primero, se carga el archivo JSON utilizando pd.read_json, teniendo en cuenta que está formado por múltiples líneas.

2. Se agrega una nueva columna al DataFrame para almacenar las fechas en formato date.

3. La columna de usuario se normaliza para extraer los datos pertinentes, ya que contiene información sobre los usuarios que realizaron las interacciones.

4. Para reducir el tamaño del DataFrame y optimizar el procesamiento, se crea un DataFrame más pequeño que incluye solo las columnas esenciales, como el ID de la interacción y la fecha, junto con los datos del usuario en columnas separadas.

5. Se utiliza groupby para crear una tabla que contenga las fechas, los usuarios y el recuento de interacciones por usuario en cada fecha.

6. Se crea una segunda tabla que calcula el total de interacciones por fecha, lo que permite ordenar la tabla por la fecha de manera efectiva.

7. Se uner la tabla del punto 5 y 6 y e ordena la tabla resultante de manera que las fechas con mayor actividad aparezcan primero, y dentro de cada fecha, los usuarios más activos se encuentren en la parte superior.

8. Se selecciona el primer registro de cada fecha, lo que proporciona el nombre de usuario con la mayor cantidad de interacciones para esa fecha en particular.

9. Finalmente, se crea una lista de tuplas que contiene las fechas y los nombres de usuario correspondientes, limitada a los primeros 10 registros para enfocarse en los usuarios más activos.

In [3]:
def q1_return_inicial(file_path: str) -> List[Tuple[datetime.date, str]]:
    # Leer el archivo JSON y cargarlo en un DataFrame
    df = pd.read_json(file_path, lines=True)
    
    # Modificar la columna de fechas
    df['date_tw'] = pd.to_datetime(df['date']).dt.date
    
    # Normalizar la columna 'user' para obtener datos de usuarios
    df_user = json_normalize(df['user'])
    
    # Unir el DataFrame original con los datos normalizados de usuarios
    df = pd.concat([df[['id','date_tw']], df_user.add_prefix('user.')], axis=1)
    
    # Agrupar por fecha y usuario, contar las interacciones y obtener el total de interacciones por fecha
    df_tw_u = df.groupby(['date_tw', 'user.username'])['id'].count().reset_index(name='count_user')
    df_tw_ug = df_tw_u.groupby('date_tw')['count_user'].sum().reset_index(name='count_total')
    
    # Unir los DataFrames y ordenar por conteo total y por usuario
    df_tw_u = pd.merge(left=df_tw_u, right=df_tw_ug, how='left', on='date_tw') \
                .sort_values(by=['count_total', 'count_user'], ascending=[False, False])
    
    # Seleccionar el usuario con más interacciones para cada fecha
    df_tw_u_f = df_tw_u.groupby('date_tw').first().reset_index().sort_values(by=['count_total'], ascending=False)
    
    # Seleccionar las primeras 10 filas y crear una lista de tuplas de fecha y usuario
    lista_resultado = list(zip(df_tw_u_f['date_tw'], df_tw_u_f['user.username']))[:10]
    
    return lista_resultado

**Resultado de ejecución del a función**

In [4]:
q1_return_inicial(file_path)

[(datetime.date(2021, 2, 12), 'RanbirS00614606'),
 (datetime.date(2021, 2, 13), 'MaanDee08215437'),
 (datetime.date(2021, 2, 17), 'RaaJVinderkaur'),
 (datetime.date(2021, 2, 16), 'jot__b'),
 (datetime.date(2021, 2, 14), 'rebelpacifist'),
 (datetime.date(2021, 2, 18), 'neetuanjle_nitu'),
 (datetime.date(2021, 2, 15), 'jot__b'),
 (datetime.date(2021, 2, 20), 'MangalJ23056160'),
 (datetime.date(2021, 2, 23), 'Surrypuria'),
 (datetime.date(2021, 2, 19), 'Preetm91')]

**Evaluación del tiempo de ejecución**

In [5]:
cProfile.run('q1_return_inicial(file_path)')

         6962513 function calls (6844681 primitive calls) in 14.689 seconds

   Ordered by: standard name

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        1    0.453    0.453   14.486   14.486 3155568970.py:1(q1_return_inicial)
        3    0.000    0.000    0.000    0.000 <__array_function__ internals>:177(all)
        1    0.000    0.000    0.001    0.001 <__array_function__ internals>:177(allclose)
       10    0.000    0.000    0.000    0.000 <__array_function__ internals>:177(append)
       11    0.000    0.000    0.001    0.000 <__array_function__ internals>:177(argsort)
        3    0.000    0.000    0.000    0.000 <__array_function__ internals>:177(array_equal)
       29    0.000    0.000    0.000    0.000 <__array_function__ internals>:177(atleast_1d)
       11    0.000    0.000    0.000    0.000 <__array_function__ internals>:177(atleast_2d)
        5    0.000    0.000    0.000    0.000 <__array_function__ internals>:177(bincount)
       35    

**Memoria total utilizada**

El ejecutar la función memory_usage((q1_return_inicial, (file_path,))), se obtiene la memoria usada durante la ejecución de la función q1_return_inicial, así que para obtener el valor total de memoria utilizada se realiza la suma de los valores de la lista. 

In [6]:
np.sum(memory_usage((q1_return_inicial, (file_path,))))

241293.609375

### **Intentos de optimización de la función inicial**

#### **Función 1**

Cambios:

1. En lugar de cargar todo el archivo JSON de una vez, se lee línea por línea para evitar cargar todo el archivo en memoria al mismo tiempo, lo que reduce el uso de memoria.

2. En lugar de usar add_prefix, se asignan los nombres de columnas directamente para mayor claridad.

3. El filtro para encontrar el usuario con mayor interaccion por fecha no se hace por método first(), si no que se usa encontrando los índices de estos usuarios y luego filtrando la tabla que contiene la información de fechas nombre de usarios, en esta misma tabla se filtran los 10 registros que pide el ejecicio.

4. Se crea un ciclo para la creación de la lista de datos requeridos. 

In [3]:
def q1_return_v1(file_path: str) -> List[Tuple[datetime.date, str]] :
    # Punto 1: Lista para almacenar los datos JSON
    data = []

    # Punto 2: Conjunto para almacenar los nombres de las claves únicas
    unique_keys = set()

    # Punto 3: Abrir el archivo JSON y leer línea por línea
    with open(file_path, 'r') as file:
        for line in file:
            # Punto 4: Cargar la línea como un objeto JSON
            json_data = json.loads(line.strip())
            
            # Punto 5: Agregar el objeto JSON a la lista
            data.append(json_data)
            
            # Punto 6: Actualizar el conjunto de claves únicas
            unique_keys.update(json_data.keys())

    # Punto 7: Crear DataFrame con los datos JSON
    df_inicial = pd.DataFrame(data).reindex(columns=unique_keys)

    # Punto 8: Modificar la columna de fechas para obtener el formato AAAA-MM-DD
    df_inicial['date_tw'] = pd.to_datetime(df_inicial['date']).dt.date
    
    # Punto 9: Crear un dataframe con los datos expandidos de los usuarios que luego se une al df inicial
    df_user = json_normalize(df_inicial['user'])
    ## Se asigna el prefijo a los nombres de las columnas para no perder el origen
    df_user.columns = ['user.'+col for col in df_user.columns ]
    ## Se une el df de usuarios el df inicial y se eliminan los ínidices
    df_inicial = pd.concat([df_inicial[['id','date_tw']].copy(),df_user],axis=1).reset_index(drop=True)   
    
    # Punto 10: Se crean groupby para procesar solo la información de fechas y nombres de usuario, agregando la columna del conteo de interacciones de usuarios
    df_tw_u = df_inicial.groupby(['date_tw','user.username'])['id'].count().reset_index(name='count_user')
    
    # Punto 11: Se crea un df nuevo que contiene solo el total de interacciones por fecha.
    df_tw_ug = df_tw_u.groupby('date_tw')['count_user'].sum().sort_values(ascending=False).reset_index(name='count_total')
    ## Se unen el df original de fechas, usuarios, conteo de interacciones por usuario al df que contiene las fechas e interacciones por fecha, se ordena de tal forma que el conteo total y por usuario es descendente
    df_tw_u = pd.merge(left=df_tw_u, right=df_tw_ug, how='left', on='date_tw').sort_values(by=['count_total','count_user'],ascending=[False,False])
    ## Se buscan los índices de la primera fila de cada fecha.
    indeices_max = df_tw_u.groupby('date_tw')['count_user'].idxmax()
    ## Se filtra del df de twits de tal forma que se ordene de la fecha de mayor interaccion a la menor, solo los primeros 10 registros
    df_tw_u_f = df_tw_u.loc[indeices_max].sort_values(by=['count_total'], ascending=False).iloc[0:10,::].reset_index(drop=True)
    
    # Punto 12: Se crea un ciclo para llenar una lista con la tuplas de fecha y el usuario con mayor cantidad de interacciones en X para ese día 
    lista_resultado = []
    for i in range(len(df_tw_u_f)):
        lista_resultado.append((df_tw_u_f['date_tw'][i],df_tw_u_f['user.username'][i]))
    
    return lista_resultado


**Resultado de ejecución del a función**

In [4]:
q1_return_v1(file_path)

[(datetime.date(2021, 2, 12), 'RanbirS00614606'),
 (datetime.date(2021, 2, 13), 'MaanDee08215437'),
 (datetime.date(2021, 2, 17), 'RaaJVinderkaur'),
 (datetime.date(2021, 2, 16), 'jot__b'),
 (datetime.date(2021, 2, 14), 'rebelpacifist'),
 (datetime.date(2021, 2, 18), 'neetuanjle_nitu'),
 (datetime.date(2021, 2, 15), 'jot__b'),
 (datetime.date(2021, 2, 20), 'MangalJ23056160'),
 (datetime.date(2021, 2, 23), 'Surrypuria'),
 (datetime.date(2021, 2, 19), 'Preetm91')]

**Evaluación del tiempo de ejecución**

In [5]:
cProfile.run('q1_return_v1(file_path)')

         8457475 function calls (8339785 primitive calls) in 11.988 seconds

   Ordered by: standard name

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        1    1.078    1.078   11.468   11.468 2760368282.py:1(q1_return_v1)
        1    0.000    0.000    0.000    0.000 2760368282.py:29(<listcomp>)
        3    0.000    0.000    0.000    0.000 <__array_function__ internals>:177(all)
        1    0.000    0.000    0.000    0.000 <__array_function__ internals>:177(allclose)
        8    0.000    0.000    0.000    0.000 <__array_function__ internals>:177(append)
       11    0.000    0.000    0.000    0.000 <__array_function__ internals>:177(argsort)
        1    0.000    0.000    0.000    0.000 <__array_function__ internals>:177(array_equal)
       11    0.000    0.000    0.000    0.000 <__array_function__ internals>:177(atleast_2d)
        4    0.000    0.000    0.000    0.000 <__array_function__ internals>:177(bincount)
       34    0.000    0.000    0.115

**Memoria total utilizada**

In [6]:
np.sum(memory_usage((q1_return_v1, (file_path,))))

87604.8828125

#### **Función 2**

Cambios:

Si bien se mantiene una estructura muy similar a la de la función 1, se realizan algunos pequeños cambios:

1. Al crear el concat entre el DataFrame inicila y el DataFrame de usuarios, en el segundo caso se usa solo la columna objetivo, que es el nombre de usuario.
2. La lista de resultados se realiza con una lista de compresión en iteración sobre las líneas del df final.

In [3]:
def q1_return_t2(file_path: str) -> List[Tuple[datetime.date, str]]:
    data = []
    unique_keys = set()

    with open(file_path, 'r') as file:
        for line in file:
            json_data = json.loads(line.strip())
            data.append(json_data)
            unique_keys.update(json_data.keys())

    df_inicial = pd.DataFrame(data).reindex(columns=unique_keys)
    df_inicial['date_tw'] = pd.to_datetime(df_inicial['date']).dt.date

    df_user = json_normalize(df_inicial['user'])
    df_user.columns = ['user.'+col for col in df_user.columns]
    df_inicial = pd.concat([df_inicial[['id','date_tw']], df_user['user.username']], axis=1).reset_index(drop=True)

    df_tw_u = df_inicial.groupby(['date_tw', 'user.username'])['id'].count().reset_index(name='count_user')
    df_tw_ug = df_tw_u.groupby('date_tw')['count_user'].sum().reset_index(name='count_total')

    df_tw_u = pd.merge(left=df_tw_u, right=df_tw_ug, how='left', on='date_tw').sort_values(by=['count_total', 'count_user'], ascending=[False, False])

    indices_max = df_tw_u.groupby('date_tw')['count_user'].idxmax()
    df_tw_u_f = df_tw_u.loc[indices_max].sort_values(by=['count_total'], ascending=False).iloc[0:10].reset_index(drop=True)

    lista_resultado = [(row['date_tw'], row['user.username']) for _, row in df_tw_u_f.iterrows()]

    return lista_resultado


**Resultado de ejecución del a función**

In [4]:
q1_return_t2(file_path)

[(datetime.date(2021, 2, 12), 'RanbirS00614606'),
 (datetime.date(2021, 2, 13), 'MaanDee08215437'),
 (datetime.date(2021, 2, 17), 'RaaJVinderkaur'),
 (datetime.date(2021, 2, 16), 'jot__b'),
 (datetime.date(2021, 2, 14), 'rebelpacifist'),
 (datetime.date(2021, 2, 18), 'neetuanjle_nitu'),
 (datetime.date(2021, 2, 15), 'jot__b'),
 (datetime.date(2021, 2, 20), 'MangalJ23056160'),
 (datetime.date(2021, 2, 23), 'Surrypuria'),
 (datetime.date(2021, 2, 19), 'Preetm91')]

**Evaluación del tiempo de ejecución**

In [5]:
cProfile.run('q1_return_t2(file_path)')

         8457048 function calls (8339342 primitive calls) in 11.876 seconds

   Ordered by: standard name

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        1    1.106    1.106   11.354   11.354 365856321.py:1(q1_return_t2)
        1    0.000    0.000    0.000    0.000 365856321.py:15(<listcomp>)
        1    0.000    0.000    0.001    0.001 365856321.py:26(<listcomp>)
        3    0.000    0.000    0.000    0.000 <__array_function__ internals>:177(all)
        1    0.000    0.000    0.000    0.000 <__array_function__ internals>:177(allclose)
        8    0.000    0.000    0.000    0.000 <__array_function__ internals>:177(append)
        9    0.000    0.000    0.000    0.000 <__array_function__ internals>:177(argsort)
        1    0.000    0.000    0.000    0.000 <__array_function__ internals>:177(array_equal)
        9    0.000    0.000    0.000    0.000 <__array_function__ internals>:177(atleast_2d)
        4    0.000    0.000    0.000    0.000 <__array_

**Memoria total utilizada**

In [6]:
np.sum(memory_usage((q1_return_t2, (file_path,))))

92975.78515625

## Punto 2 emojies

### **Función inicial**

**Consideraciones con las que se creó la función**

1. Primero, se carga el archivo JSON utilizando pd.read_json, teniendo en cuenta que está formado por múltiples líneas. La forma de carga adoptada tiene el sentido de no sobrecargar la memoria con una carga completa.

2. Las claves únicas tiene el sentido de asegurar que al conformar el dataframe luego de leer los datos van a estar encabezados de forma adecuada.

3. Se realiza el uso de la librería emoji y dentro de ella el atriburo EMOJI_DATA, el cual contiene la información asociada a los emojies.

4. Se crea una función para extraer los emojies de cada fila de contenido, de esta manera crea una lista de emojies para cada fila de content.

5. Se crea una columna dentro del dataFrame para poder almancenas los emojies encontrados en cada fila del content.

6. Se crea una lista de vacía de emojies a la cual se va integrando con las listas de los emojies encontrados en cada fila de la colimna 'emojis'

7. Se crea un dataframe con la lista de los emojies, luego se crea un nuevo df sobre este luego de hacer el conteo descente de aparición para cada emojie.

8. Se filtran los datos de color de los emojies, dado que por el momento, el enfoque se centra en identificar la forma básica de los emojis y no en considerar sus modificadores de color. Esto puede ayudar a evitar confusiones sobre el alcance y los objetivos del análisis actual.

9. Finalmente, se crea una lista de tuplas que contiene los emojies y la cantidad de apariciones, limitada a los primeros 10 registros para enfocarse en los usuarios más activos.

In [3]:
def q2_run_inicial(file_path: str) -> List[Tuple[str, int]]:
    # Leer el archivo JSON y cargarlo en un DataFrame
    data = []
    # Crear conjunto para la claves únicas 
    unique_keys = set()

    with open(file_path, 'r') as file:
        for line in file:
            json_data = json.loads(line.strip())
            data.append(json_data)
            unique_keys.update(json_data.keys())

    #Crear el dataframe inicial en función de los datos leídos y con columnas nombradas tal como las claves únicas obtenidas
    df_inicial = pd.DataFrame(data).reindex(columns=unique_keys)
    
    #Usando la librería de emoji se crea una lista con los emojies "c" dentro un texto que se pasa, a la función
    def extraer_emojis(texto):
        return [c for c in texto if c in emoji.EMOJI_DATA]

    #Se aplica la función sobre la columna de contenido y se crea una nueva con el fin de procesar los emojies posteriormente
    df_inicial['emojis'] = df_inicial['content'].apply(extraer_emojis)
    
    lista_emojies = []

    #Se crea un ciclo para obtener todos los emojies en una sola lista.
    for fila in df_inicial['emojis']:
        lista_emojies.extend(fila)
        
    df_emojies = pd.DataFrame(lista_emojies,columns=['emojie'])
    
    df_ef = pd.DataFrame(df_emojies.value_counts(),columns=['Cuenta']).reset_index()
    
    #Se filtran los símbolos que representan la modificación de color de los emojies, se crea esta función bajo el supuesto que por ahora solo se necesita la figura del emojie no su color.
    df_ef = df_ef[~df_ef['emojie'].isin(['🏻','🏽','🏼','🏾','🏿'])][0:10]
    
    lista_resultado = [(row['emojie'], row['Cuenta']) for _, row in df_ef.iterrows()]

    return lista_resultado


**Resultado de ejecución del a función**

In [4]:
q2_run_inicial(file_path)

[('🙏', 7286),
 ('😂', 3072),
 ('🚜', 2972),
 ('✊', 2411),
 ('🌾', 2363),
 ('❤', 1779),
 ('🤣', 1668),
 ('👇', 1108),
 ('💚', 1040),
 ('💪', 947)]

**Evaluación del tiempo de ejecución**

In [5]:
cProfile.run('q2_run_inicial(file_path)')

         2338463 function calls (2338339 primitive calls) in 8.892 seconds

   Ordered by: standard name

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        1    0.895    0.895    8.335    8.335 2406996805.py:1(q2_run_inicial)
   117407    0.031    0.000    1.135    0.000 2406996805.py:17(extraer_emojis)
   117407    1.104    0.000    1.104    0.000 2406996805.py:18(<listcomp>)
        1    0.000    0.000    0.001    0.001 2406996805.py:36(<listcomp>)
        4    0.000    0.000    0.000    0.000 <__array_function__ internals>:177(append)
        2    0.000    0.000    0.000    0.000 <__array_function__ internals>:177(argsort)
        1    0.000    0.000    0.000    0.000 <__array_function__ internals>:177(array_equal)
        2    0.000    0.000    0.000    0.000 <__array_function__ internals>:177(atleast_2d)
        3    0.000    0.000    0.000    0.000 <__array_function__ internals>:177(bincount)
        9    0.000    0.000    0.026    0.003 <__array_fun

**Memoria total utilizada**

In [6]:
np.sum(memory_usage((q2_run_inicial, (file_path,))))

68143.96875

### **Intentos de optimización de la función inicial**

### **Función 1**

En este caso se usa la misma estructura de la función anterior, pero se mejora la escrutura de la creación del data frame con los emojis y el recuento de sus apariciones

In [3]:
def q2_run_v1(file_path: str) -> List[Tuple[str, int]]:
    # Leer el archivo JSON y cargarlo en un DataFrame
    data = []
    # Crear conjunto para la claves únicas 
    unique_keys = set()

    with open(file_path, 'r') as file:
        for line in file:
            json_data = json.loads(line.strip())
            data.append(json_data)
            unique_keys.update(json_data.keys())

    #Crear el dataframe inicial en función de los datos leídos y con columnas nombradas tal como las claves únicas obtenidas
    df_inicial = pd.DataFrame(data).reindex(columns=unique_keys)
    
    #Usando la librería de emoji se crea una lista con los emojies "c" dentro un texto que se pasa, a la función
    def extraer_emojis(texto):
        return [c for c in texto if c in emoji.EMOJI_DATA]

    #Se aplica la función sobre la columna de contenido y se crea una nueva con el fin de procesar los emojies posteriormente
    df_inicial['emojis'] = df_inicial['content'].apply(extraer_emojis)
    
    lista_emojies = []

    #Se crea un ciclo para obtener todos los emojies en una sola lista.
    for fila in df_inicial['emojis']:
        lista_emojies.extend(fila)

    df_ef = pd.DataFrame(pd.DataFrame(lista_emojies,columns=['emojie']).value_counts(),columns=['Cuenta']).reset_index()
    
    #Se filtran los símbolos que representan la modificación de color de los emojies, se crea esta función bajo el supuesto que por ahora solo se necesita la figura del emojie no su color.
    df_ef = df_ef[~df_ef['emojie'].isin(['🏻','🏽','🏼','🏾','🏿'])][0:10]
    
    lista_resultado = [(row['emojie'], row['Cuenta']) for _, row in df_ef.iterrows()]

    return lista_resultado


**Resultado de ejecución del a función**

In [4]:
q2_run_v1(file_path)

[('🙏', 7286),
 ('😂', 3072),
 ('🚜', 2972),
 ('✊', 2411),
 ('🌾', 2363),
 ('❤', 1779),
 ('🤣', 1668),
 ('👇', 1108),
 ('💚', 1040),
 ('💪', 947)]

**Evaluación del tiempo de ejecución**

In [5]:
cProfile.run('q2_run_v1(file_path)')

         2338463 function calls (2338339 primitive calls) in 9.060 seconds

   Ordered by: standard name

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        1    0.910    0.910    8.406    8.406 3950940001.py:1(q2_run_v1)
   117407    0.033    0.000    1.144    0.000 3950940001.py:17(extraer_emojis)
   117407    1.110    0.000    1.110    0.000 3950940001.py:18(<listcomp>)
        1    0.000    0.000    0.001    0.001 3950940001.py:34(<listcomp>)
        4    0.000    0.000    0.000    0.000 <__array_function__ internals>:177(append)
        2    0.000    0.000    0.000    0.000 <__array_function__ internals>:177(argsort)
        1    0.000    0.000    0.000    0.000 <__array_function__ internals>:177(array_equal)
        2    0.000    0.000    0.000    0.000 <__array_function__ internals>:177(atleast_2d)
        3    0.000    0.000    0.000    0.000 <__array_function__ internals>:177(bincount)
        9    0.000    0.000    0.025    0.003 <__array_function

**Memoria total utilizada**

In [6]:
np.sum(memory_usage((q2_run_v1, (file_path,))))

78002.734375

### **Función 2**

En esta función se cambia la manera en la que se presenta el dataframe con los emojis y su contero de aparición, al tiempo que la creación de la lista de emojis pasa de ser un ciclo a una lista de compresión.

In [7]:
def q2_run_v2(file_path: str) -> List[Tuple[str, int]]:
    # Leer el archivo JSON y cargarlo en un DataFrame
    data = []
    # Crear conjunto para la claves únicas 
    unique_keys = set()

    with open(file_path, 'r') as file:
        for line in file:
            json_data = json.loads(line.strip())
            data.append(json_data)
            unique_keys.update(json_data.keys())

    #Crear el dataframe inicial en función de los datos leídos y con columnas nombradas tal como las claves únicas obtenidas
    df_inicial = pd.DataFrame(data).reindex(columns=unique_keys)
    
    #Usando la librería de emoji se crea una lista con los emojies "c" dentro un texto que se pasa, a la función
    def extraer_emojis(texto):
        return [c for c in texto if c in emoji.EMOJI_DATA]

    #Se aplica la función sobre la columna de contenido y se crea una nueva con el fin de procesar los emojies posteriormente
    df_inicial['emojis'] = df_inicial['content'].apply(extraer_emojis)
    
    # Crear una lista plana con todos los emojis encontrados en todas las filas
    lista_emojies = [emoji for fila in df_inicial['emojis'] for emoji in fila]
    
    # Crear DataFrame con los emojis y su frecuencia
    df_ef = pd.DataFrame(Counter(lista_emojies).most_common(), columns=['emojie', 'Cuenta'])
        
    #Se filtran los símbolos que representan la modificación de color de los emojies, se crea esta función bajo el supuesto que por ahora solo se necesita la figura del emojie no su color.
    df_ef = df_ef[~df_ef['emojie'].isin(['🏻','🏽','🏼','🏾','🏿'])][0:10]
    
    lista_resultado = [(row['emojie'], row['Cuenta']) for _, row in df_ef.iterrows()]

    return lista_resultado

**Resultado de ejecución del a función**

In [4]:
q2_run_v2(file_path)

[('🙏', 7286),
 ('😂', 3072),
 ('🚜', 2972),
 ('✊', 2411),
 ('🌾', 2363),
 ('❤', 1779),
 ('🤣', 1668),
 ('👇', 1108),
 ('💚', 1040),
 ('💪', 947)]

**Evaluación del tiempo de ejecución**

In [5]:
cProfile.run('q2_run_v2(file_path)')

         2219117 function calls (2219026 primitive calls) in 9.071 seconds

   Ordered by: standard name

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        1    0.870    0.870    8.477    8.477 894466136.py:1(q2_run_v2)
   117407    0.039    0.000    1.308    0.000 894466136.py:17(extraer_emojis)
   117407    1.269    0.000    1.269    0.000 894466136.py:18(<listcomp>)
        1    0.010    0.010    0.010    0.010 894466136.py:24(<listcomp>)
        1    0.000    0.000    0.001    0.001 894466136.py:32(<listcomp>)
        2    0.000    0.000    0.000    0.000 <__array_function__ internals>:177(append)
        2    0.000    0.000    0.000    0.000 <__array_function__ internals>:177(argsort)
        2    0.000    0.000    0.000    0.000 <__array_function__ internals>:177(atleast_2d)
        1    0.000    0.000    0.000    0.000 <__array_function__ internals>:177(bincount)
        6    0.000    0.000    0.027    0.004 <__array_function__ internals>:177(concat

**Memoria total utilizada**

In [6]:
np.sum(memory_usage((q2_run_v2, (file_path,))))

67479.74609375

## Punto 3 influyentes

### **Análisis de mención a los usuarios**

Se evidencia que tanto en la columna content como en la columna mentionedUsers hat mención a los usuarios, por tanto, es necesario realizar la exploración aleatoria de si los usuarios mencionados en content con los mismo de mentionedUsers

In [3]:
def qinicial(file_path: str) -> List[Tuple[str, int]]:
    # Leer el archivo JSON y cargarlo en un DataFrame
    data = []
    # Crear conjunto para la claves únicas 
    unique_keys = set()

    with open(file_path, 'r') as file:
        for line in file:
            json_data = json.loads(line.strip())
            data.append(json_data)
            unique_keys.update(json_data.keys())

    #Crear el dataframe inicial en función de los datos leídos y con columnas nombradas tal como las claves únicas obtenidas
    df_inicial = pd.DataFrame(data).reindex(columns=unique_keys)

    return df_inicial

#Se almacena el resultado en un df sin explotar ni expandir para realizar un adecuado análisis comparativo
resultado1 = qinicial(file_path)
#Se almacena el resulta en un df con los array expandidos en filas
resultado2 = qinicial(file_path)[['content','mentionedUsers']].copy().explode(column = 'mentionedUsers')
#resultado2= resultado2[~resultado2['mentionedUsers'].isnull()].reset_index()
pd.set_option('display.max_colwidth', 500) 

display(resultado2)

,content,mentionedUsers
0,The world progresses while the Indian police and Govt are still trying to take India back to the horrific past through its tyranny. \n\n@narendramodi @DelhiPolice Shame on you. \n\n#ModiDontSellFarmers \n#FarmersProtest \n#FreeNodeepKaur https://t.co/es3kn0IQAF,"{'username': 'narendramodi', 'displayname': 'Narendra Modi', 'id': 18839785, 'description': None, 'rawDescription': None, 'descriptionUrls': None, 'verified': None, 'created': None, 'followersCount': None, 'friendsCount': None, 'statusesCount': None, 'favouritesCount': None, 'listedCount': None, 'mediaCount': None, 'location': None, 'protected': None, 'linkUrl': None, 'linkTcourl': None, 'profileImageUrl': None, 'profileBannerUrl': None, 'url': 'https://twitter.com/narendramodi'}"
0,The world progresses while the Indian police and Govt are still trying to take India back to the horrific past through its tyranny. \n\n@narendramodi @DelhiPolice Shame on you. \n\n#ModiDontSellFarmers \n#FarmersProtest \n#FreeNodeepKaur https://t.co/es3kn0IQAF,"{'username': 'DelhiPolice', 'displayname': '#DilKiPolice Delhi Police', 'id': 1850705408, 'description': None, 'rawDescription': None, 'descriptionUrls': None, 'verified': None, 'created': None, 'followersCount': None, 'friendsCount': None, 'statusesCount': None, 'favouritesCount': None, 'listedCount': None, 'mediaCount': None, 'location': None, 'protected': None, 'linkUrl': None, 'linkTcourl': None, 'profileImageUrl': None, 'profileBannerUrl': None, 'url': 'https://twitter.com/DelhiPolice'}"
1,"#FarmersProtest \n#ModiIgnoringFarmersDeaths \n#ModiDontSellFarmers \n@Kisanektamorcha \nFarmers constantly distroying crops throughout India. \nReally, it's hearts breaking...we care about our crops like our children. And govt. agriculture minister is laughing on us🚜🌾WE WILL WIN💪 https://t.co/kLspngG9xE","{'username': 'Kisanektamorcha', 'displayname': 'Kisan Ekta Morcha', 'id': 1338536920066879488, 'description': None, 'rawDescription': None, 'descriptionUrls': None, 'verified': None, 'created': None, 'followersCount': None, 'friendsCount': None, 'statusesCount': None, 'favouritesCount': None, 'listedCount': None, 'mediaCount': None, 'location': None, 'protected': None, 'linkUrl': None, 'linkTcourl': None, 'profileImageUrl': None, 'profileBannerUrl': None, 'url': 'https://twitter.com/Kisanekt..."
2,ਪੈਟਰੋਲ ਦੀਆਂ ਕੀਮਤਾਂ ਨੂੰ ਮੱਦੇਨਜ਼ਰ ਰੱਖਦੇ ਹੋਏ \nਮੇਰੇ ਹਿਸਾਬ ਨਾਲ ਬਾਹਰ(ਪ੍ਰਦੇਸ਼) ਜਾਣ ਨਾਲੋਂ ਬਿਹਤਰ ਆ ਭਾਰਤ 'ਚ ਪੈਟਰੋਲ ਪੰਪ ਪਾ ਲਈਏ। 🤫🤫🤔🤔\n#FarmersProtest,None
3,@ReallySwara @rohini_sgh watch full video here https://t.co/wBPNdJdB0n\n#farmersprotest #NoFarmersNoFood https://t.co/fUsTOKOcXK,"{'username': 'ReallySwara', 'displayname': 'Swara Bhasker', 'id': 1546101560, 'description': None, 'rawDescription': None, 'descriptionUrls': None, 'verified': None, 'created': None, 'followersCount': None, 'friendsCount': None, 'statusesCount': None, 'favouritesCount': None, 'listedCount': None, 'mediaCount': None, 'location': None, 'protected': None, 'linkUrl': None, 'linkTcourl': None, 'profileImageUrl': None, 'profileBannerUrl': None, 'url': 'https://twitter.com/ReallySwara'}"
...,...,...
117402,#FarmersProtest #KisanAndolan #KisaanMajdoorEktaZindabad,None
117403,PM मोदी की अपील के बीच संयुक्त किसान मोर्चा का बहुत बड़ा ऐलान | PunjabTak\n#FarmersProtest #KisanAandolan #FarmersFury #Chakkajam #SanyukthKisanMorcha #MasalJulus #NarendraModi #DarshanPal #NarendraSinghTomar #YogendraYadav #TollFree #CandleMarch \n\nhttps://t.co/AzZNOGI8BX https://t.co/1UZvVvYK2e,None
117404,United we stand.\nDivided we fall\n#MahapanchayatRevolution\n#FarmersProtest https://t.co/LPJkjxldM2,None
117405,"सिंघु बॉर्डर पर लंबी लड़ाई की तैयारी, किसानों ने प्रदर्शन स्थल पर बुनियादी ढांचे को मजबूत करना किया शुरू!\n\n#FarmersProtest | #MSP | #FarmBill2020 | #SinghuBorder\nhttps://t.co/bkjh7WXc0w",None


Se identifican el total de @ que hay en la columna content

In [4]:
#Estos @ son los posible etiquetados
(resultado1['content'].str.count('@').sum()) - (resultado1['content'].str.count('@@').sum())

104215

Se realiza la identificación fila a fila de la cantidad de @

In [5]:
resultado1['content'].str.count('@')

0         2
1         1
2         0
3         2
4         0
         ..
117402    0
117403    0
117404    0
117405    0
117406    1
Name: content, Length: 117407, dtype: int64

Se identica la cantidad de usuario en cada fila de la columna mentionedUsers, esto para luego compararlo con la cantidad de @ en el contenido

In [6]:
resultado1['mentionedUsers'].apply(lambda x: len(x) if x is not None else 0)

0         2
1         1
2         0
3         2
4         0
         ..
117402    0
117403    0
117404    0
117405    0
117406    1
Name: mentionedUsers, Length: 117407, dtype: int64

In [7]:
resultado1['mentionedUsers'].apply(lambda x: len(x) if x is not None else 0).sum()

103403

Se filtra el df de tal forma que se puedan observar en el primer caso todas las filas donde la cantidad de @ en el contenido no conincide con la cantidad de usiarios en la columna mentionedUsers

In [8]:
resultado1[(resultado1['content'].str.count('@') != resultado1['mentionedUsers'].apply(lambda x: len(x) if x is not None else 0))][['content','mentionedUsers']]

,content,mentionedUsers
222,♨️♨️ AJIO : Women's Western Wear upto 85% Off starting @ 90\n\nhttps://t.co/fM9Y6Sh5ix\n\n#nseindia #nifty50 #RajasthanBudget #RubinaDilaik #Motera #MoteraTestMatch #MoteraCricketStadium #ForaMamacita #FarmersProtest #HBDAmma #JUNGKOOK #noitsnot #zerodha #INDvENG #Budget2021 #BB14,None
580,#SFJ #ArrestPannu Launches Portal\nhttps://t.co/739DRB6mA4\nEmail\nDeclare Independence Of #Bengal #Maharashtra \n#FarmersProtest\n #KisaanHulKhalistan \n@iamnaveenkapoor @Ashoke_Raj @januamyth @abhishekjha157 @AshishSinghLIVE @rishhikesh @ArunSharmaLive \n https://t.co/PBHewoymn1,"[{'username': 'IamNaveenKapoor', 'displayname': 'Naveen Kapoor ANI', 'id': 253512688, 'description': None, 'rawDescription': None, 'descriptionUrls': None, 'verified': None, 'created': None, 'followersCount': None, 'friendsCount': None, 'statusesCount': None, 'favouritesCount': None, 'listedCount': None, 'mediaCount': None, 'location': None, 'protected': None, 'linkUrl': None, 'linkTcourl': None, 'profileImageUrl': None, 'profileBannerUrl': None, 'url': 'https://twitter.com/IamNaveenKapoor'}..."
907,#DidiWithCOVIDWarriors ❤️\n#ModiAgainstFarmers \n#FarmersProtest \n#DishaRavi \n@NilanjanDas_ @ItsYourDev @aitcsudip @aitcsupriya @LakhendraMeena @EaktaKisan https://t.co/HdyHiAgK8e https://t.co/0E85rkr8mD,"[{'username': 'NilanjanDas_', 'displayname': 'Nilanjan Das', 'id': 763036361460965376, 'description': None, 'rawDescription': None, 'descriptionUrls': None, 'verified': None, 'created': None, 'followersCount': None, 'friendsCount': None, 'statusesCount': None, 'favouritesCount': None, 'listedCount': None, 'mediaCount': None, 'location': None, 'protected': None, 'linkUrl': None, 'linkTcourl': None, 'profileImageUrl': None, 'profileBannerUrl': None, 'url': 'https://twitter.com/NilanjanDas_'}, ..."
1332,"देश की आन बान शान है, सच के लिए लड़ रहा किसान है.....\nकिसान है तो देश है। \n#FarmersProtest #StandWithFarmers \n@DeependerSHooda @TeamDeepender @RakeshTikaitBKU @GurunamchuduniBKU https://t.co/3Vzs5PAj9W","[{'username': 'DeependerSHooda', 'displayname': 'Deepender S Hooda', 'id': 49375859, 'description': None, 'rawDescription': None, 'descriptionUrls': None, 'verified': None, 'created': None, 'followersCount': None, 'friendsCount': None, 'statusesCount': None, 'favouritesCount': None, 'listedCount': None, 'mediaCount': None, 'location': None, 'protected': None, 'linkUrl': None, 'linkTcourl': None, 'profileImageUrl': None, 'profileBannerUrl': None, 'url': 'https://twitter.com/DeependerSHooda'},..."
1494,"#FarmersProtest #StandWithFarmers #AskIndiaWhy @PMOIndia, @nstomar @thepmo, @UNDP, @UNFAOand @PMOIndia, @UNDP_India, @nstomar, @TimesTrolley","[{'username': 'PMOIndia', 'displayname': 'PMO India', 'id': 471741741, 'description': None, 'rawDescription': None, 'descriptionUrls': None, 'verified': None, 'created': None, 'followersCount': None, 'friendsCount': None, 'statusesCount': None, 'favouritesCount': None, 'listedCount': None, 'mediaCount': None, 'location': None, 'protected': None, 'linkUrl': None, 'linkTcourl': None, 'profileImageUrl': None, 'profileBannerUrl': None, 'url': 'https://twitter.com/PMOIndia'}, {'username': 'nstoma..."
...,...,...
117185,#BJPGovtDictatingTwitter #MahapanchayatRevolution #FreeNodeepKaur #FarmersAboveReligiousHate #freedeepsidhunow #FreeDeepSidhu #SpeakupforNodeepkaur #FarmersProtest #isupportfarmersprotest #RihannaSupportsIndianFarmers #Anti_Farmer_Bollywood #BJPdestroysDemocracy #Canada ਪੰਜਾਬ@@ https://t.co/z9JIk829xp,None
117203,United \nStrong \nTogether \nThis is New Revolution \n\nMAHAPANCHAYAT REVOLUTION \n#आज_किसान_टोल_फ्री_करायेगे\n#MahapanchayatRevolution \n#FarmersProtest\n@RakeshTikaitBKU\n@charuni_BKU\n@Tractor2twitr @Kisanektamorcha\n@ppbajpai @Mani_KaurRai @IamProudPunjabi\n https://t.co/m8V0QCVjzk,"[{'username': 'RakeshTikaitBKU', 'displayname': 'Rakesh Tikait', 'id': 831948775165923330, 'description': None, 'rawDescription': None, 'descriptionUrls': None, 'verified': None, 'created': None, 'fo

Acá se pueden ver casos como este ਪੰਜਾਬ@@, en el que hay doble @ por lo que se puede hacer un conteo errado, aparte de ello, se observan situaciones como la presentada donde el usuario posiblemente esté indicado antes del @ dadas las condiciones de gramáticas propias de la lengua

** Ahora se muestran solo los casos en los que en verdad no se identifica ningún usuario

In [9]:
resultado1[(resultado1['content'].str.count('@') != resultado1['mentionedUsers'].apply(lambda x: len(x) if x is not None else 0)) & (resultado1['mentionedUsers'].isnull())][['content','mentionedUsers']]

,content,mentionedUsers
222,♨️♨️ AJIO : Women's Western Wear upto 85% Off starting @ 90\n\nhttps://t.co/fM9Y6Sh5ix\n\n#nseindia #nifty50 #RajasthanBudget #RubinaDilaik #Motera #MoteraTestMatch #MoteraCricketStadium #ForaMamacita #FarmersProtest #HBDAmma #JUNGKOOK #noitsnot #zerodha #INDvENG #Budget2021 #BB14,None
3801,#FarmersProtest@ https://t.co/hPZLpNAcVI,None
3854,"Indian Government warning notice @ TikriBorder, one of the farmers protest site. Check out the standard of language used ""Titar Vitar"" means ""Get lost, Just shove off"" ?? \nHow pathetic!! \n\n#FarmersProtest https://t.co/ZfVitCb78V",None
4394,Please@DelhiPolice #FreePieterFriedrich #FarmersProtests #FarmersProtest https://t.co/oGgmAHlp2X,None
5754,Learned Experience: Life is so Beautiful.Just Live it do not Just.Enjoy small small things. Be with your Family and friends. Ultimately that matters only. \nPS: Time changes . Be Strong and Be a Fighter. #FarmersProtest #PositiveVibesOnly. ਨਜੱਰੀਆ ਬਦਲੋ. Vc @ Baba Nirala https://t.co/hdPeNaaxSi,None
...,...,...
115540,@dhammusaab ਜਿਨ੍ਨੇ ਵੀ ਪੰਜਾਬੀ ਏ ਟਵ੍ੀਟ ਵੇਖ ਰਹੇ \nਪਲੀਜ ਫੌਲੋ ਕਰੋ !!\nਜਿਨੇ ਜਿਆਦਾ ਅਸੀ ਇਕ ਦੂਜੇ ਨੂ ਫੌਲੋ ਕਰਾਂਗੇ trend ਚ ਦੁਗਨੀ speed ਆਊਗੀ !!\n\nकिसान मज़दूर एकता जिन्दाबाद !!\n\nਕਿਸਾਨ ਮਜਦੂਰ ਏਕਤਾ ਜਿੰਦਾਬਾਦ !!\n\n#MahapanchayatRevolution\n#FarmersProtest \n#किसान_एकता_जिंदाबाद \n#ਕਿਸਾਨ_ਮਜਦੂਰ_ਏਕਤਾ_ਜਿੰਦਾਬਾਦ,None
116305,"चच्चा, ओ B@#$ वाले चच्चा \n""50 दिन में अगर आपके सपनों का भारत न दूँ तो चौराहे पे ज़िंदा जला देना"" वाला वादा याद है कि नहीं? \n#PromiseDay\n#FarmersProstests #GodiMedia #FarmersProtests #FarmersProtest #india #farming #किसानो_अब_रेल_रोको भारतीय जनसंघ #farmersgenocide",None
117185,#BJPGovtDictatingTwitter #MahapanchayatRevolution #FreeNodeepKaur #FarmersAboveReligiousHate #freedeepsidhunow #FreeDeepSidhu #SpeakupforNodeepkaur #FarmersProtest #isupportfarmersprotest #RihannaSupportsIndianFarmers #Anti_Farmer_Bollywood #BJPdestroysDemocracy #Canada ਪੰਜਾਬ@@ https://t.co/z9JIk829xp,None
117315,#BJPGovtDictatingTwitter #MahapanchayatRevolution #FreeNodeepKaur #FarmersAboveReligiousHate #freedeepsidhunow #FreeDeepSidhu #SpeakupforNodeepkaur #FarmersProtest #isupportfarmersprotest #RihannaSupportsIndianFarmers #Anti_Farmer_Bollywood #BJPdestroysDemocracy #Canada ਪੰਜਾਬ@@ https://t.co/SQZlCwjC4k,None


A continuación la cantidad de menciones en la columna dispuesta para ello

In [10]:
len(resultado2)-resultado2['mentionedUsers'].isnull().sum()

103403

Al observar el caso anterior se deja en evidencia casos como los de los registros de las fila 117388, el cual tiene indicados dos veces a la misma persona, en este caso se debería poder entender la necesidad del cliente final, dado que una peersona comentada dos veces en un mismo twit puede indicar que la persona es relevante para quien actual en "X", pero al tiempo podría mostrar un dato desviado, dado que si solo queremos conocer a quien se han etiquetado por comentario, pues no importa su número de veces etiquetado, se deberá contar una única vez.

Por el momento, aludiendo a que si hay una etiqueta múltiple para una misma cuenta en un solo comentario podría indicar un sentido de relevancia para quien redacta el mensaje, se trasladará esa urgencia al análisis de los más etiquetados, de tal forma que se obseve en la lista la urgencia o importancia que le dan los usuario de "X" a quienes etiquetan. 

### **Función inicial**

Dado la explicación anterior se va a hacer uso de la columna mentionedUsers para obtener la información de la mensión de usuarios dentro del contenido de una publicación, cabe resaltar dos aspectos, la diferencia total de posibles usuarios verdaderos etiquetados es de 812, de los cuales solo un total de 218 no son listados en la columna mentionedUsers. Se va a asumir que los usuarios realmente mencionados están en la columna mentionedUsers, en caso tal de requerir la identificación detallada, se procedería al uso de expresiones para identificar los usuarios.

La creación de esta función, tiene en cuenta:

1. Primero, se carga el archivo JSON utilizando pd.read_json, teniendo en cuenta que está formado por múltiples líneas. La forma de carga adoptada tiene el sentido de no sobrecargar la memoria con una carga completa.

2. Las claves únicas tiene el sentido de asegurar que al conformar el dataframe luego de leer los datos van a estar encabezados de forma adecuada.

3. Se crea el df con todad la información de JSON y las columnas identificadas.

4. Se crea un dataframe inicial, para ello se crea un dataFrame de solo una columna, la columna mentionedUsers, que es donde se linstan los usuarios que se mensionan en los comentarios

5. Se crea un dataFrame de usarios relevantes para ello se extienden en filas los array de la columna mentionedUsers y luego se expanden en columnas las varaibles de los diccionarios.

6. Se crea un nuevo dataFrame con una sola columna de nombres de usarios y una columna del conteo.

7. Finalmente, se crea una lista de tuplas que contiene los emojies y la cantidad de apariciones, limitada a los primeros 10 registros para enfocarse en los usuarios más activos.


In [3]:
def q3_run_inicial(file_path: str) -> List[Tuple[str, int]]:
    # Leer el archivo JSON y cargarlo en un DataFrame
    data = []
    # Crear conjunto para la claves únicas 
    unique_keys = set()

    with open(file_path, 'r') as file:
        for line in file:
            json_data = json.loads(line.strip())
            data.append(json_data)
            unique_keys.update(json_data.keys())

    #Crear el dataframe inicial en función de los datos leídos y con columnas nombradas tal como las claves únicas obtenidas
    df_inicial = pd.DataFrame(data).reindex(columns=unique_keys)
    
    #Dado que se ha validado previamente que no hay twits duplicados, se puede usar la columna mentionedUsers, para ser explotada (explode) y luego expandida (json_normalize) y con esta hacer todo el procesamiento solicitado. Por otro lado, antes de explotar y expandir
    
    df_inicial = pd.DataFrame(df_inicial['mentionedUsers'])
    
    df_relevantes =  json_normalize(df_inicial.explode(column='mentionedUsers').reset_index(drop=True)['mentionedUsers'])
    
    # Crear DataFrame con los usuarios relevantes y su frecuencia
    df_relev = pd.DataFrame(Counter(df_relevantes['username']).most_common(), columns=['username', 'Cuenta'])
    
    #Se creas lista con las tuplas de nombre de uduario y el conteo de apareiocnes y se filtra para dejar solo los primeros 10 registros.
    lista_resultado = [(row['username'], row['Cuenta']) for _, row in df_relev.iterrows()][:10]
    
    return lista_resultado

**Resultado de ejecución del a función**

In [4]:
q3_run_inicial(file_path)

[(nan, 79373),
 ('narendramodi', 2265),
 ('Kisanektamorcha', 1840),
 ('RakeshTikaitBKU', 1644),
 ('PMOIndia', 1427),
 ('RahulGandhi', 1146),
 ('GretaThunberg', 1048),
 ('RaviSinghKA', 1019),
 ('rihanna', 986),
 ('UNHumanRights', 962)]

**Evaluación del tiempo de ejecución**

In [5]:
cProfile.run('q3_run_inicial(file_path)')

         10252248 function calls (10038867 primitive calls) in 10.955 seconds

   Ordered by: standard name

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        1    0.988    0.988   10.547   10.547 370186207.py:1(q3_run_inicial)
        1    0.020    0.020    0.964    0.964 370186207.py:26(<listcomp>)
        3    0.000    0.000    0.000    0.000 <__array_function__ internals>:177(argsort)
        3    0.000    0.000    0.000    0.000 <__array_function__ internals>:177(atleast_2d)
        7    0.000    0.000    0.037    0.005 <__array_function__ internals>:177(concatenate)
       58    0.000    0.000    0.001    0.000 <__array_function__ internals>:177(copyto)
        1    0.000    0.000    0.000    0.000 <__array_function__ internals>:177(delete)
        3    0.000    0.000    0.037    0.012 <__array_function__ internals>:177(vstack)
       14    0.000    0.000    0.000    0.000 <frozen importlib._bootstrap>:1053(_handle_fromlist)
        1    0.407    0.4

**Memoria total utilizada**

In [6]:
np.sum(memory_usage((q3_run_inicial, (file_path,))))

72781.48046875

### **Intentos de optimización de la función inicial**

### **Función 1**

El mayor cambio en esta función está dado en que se eliminan las filas vacías de la columna mentionedUsers para no tener que operar con estos datos.

In [3]:
def q3_run_v1(file_path: str) -> List[Tuple[str, int]]:
    # Leer el archivo JSON y cargarlo en un DataFrame
    data = []
    # Crear conjunto para la claves únicas 
    unique_keys = set()

    with open(file_path, 'r') as file:
        for line in file:
            json_data = json.loads(line.strip())
            data.append(json_data)
            unique_keys.update(json_data.keys())

    #Crear el dataframe inicial en función de los datos leídos y con columnas nombradas tal como las claves únicas obtenidas
    df_inicial = pd.DataFrame(data).reindex(columns=unique_keys)
    
    #Dado que se ha validado previamente que no hay twits duplicados, se puede usar la columna mentionedUsers, para ser explotada (explode) y luego expandida (json_normalize) y con esta hacer todo el procesamiento solicitado. Por otro lado, antes de explotar y expandir.
    
    df_relevantes =  json_normalize(pd.DataFrame(df_inicial['mentionedUsers']).dropna().explode(column='mentionedUsers').reset_index(drop=True)['mentionedUsers'])
    
    # Crear DataFrame con los usuarios relevantes y su frecuencia
    df_relev = pd.DataFrame(Counter(df_relevantes['username']).most_common(), columns=['username', 'Cuenta'])
    
    #Se creas lista con las tuplas de nombre de uduario y el conteo de apareiocnes y se filtra para dejar solo los primeros 10 registros.
    lista_resultado = [(row['username'], row['Cuenta']) for _, row in df_relev.iterrows()][:10]
    
    return lista_resultado

**Resultado de ejecución del a función**

In [4]:
q3_run_v1(file_path)

[('narendramodi', 2265),
 ('Kisanektamorcha', 1840),
 ('RakeshTikaitBKU', 1644),
 ('PMOIndia', 1427),
 ('RahulGandhi', 1146),
 ('GretaThunberg', 1048),
 ('RaviSinghKA', 1019),
 ('rihanna', 986),
 ('UNHumanRights', 962),
 ('meenaharris', 926)]

**Evaluación del tiempo de ejecución**

In [5]:
cProfile.run('q3_run_v1(file_path)')

         9856895 function calls (9722860 primitive calls) in 10.621 seconds

   Ordered by: standard name

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        1    1.042    1.042   10.057   10.057 2117970924.py:1(q3_run_v1)
        1    0.020    0.020    0.962    0.962 2117970924.py:24(<listcomp>)
        1    0.000    0.000    0.000    0.000 <__array_function__ internals>:177(all)
        3    0.000    0.000    0.000    0.000 <__array_function__ internals>:177(argsort)
        3    0.000    0.000    0.000    0.000 <__array_function__ internals>:177(atleast_2d)
        7    0.000    0.000    0.046    0.007 <__array_function__ internals>:177(concatenate)
       59    0.000    0.000    0.000    0.000 <__array_function__ internals>:177(copyto)
        1    0.000    0.000    0.000    0.000 <__array_function__ internals>:177(delete)
        1    0.000    0.000    0.000    0.000 <__array_function__ internals>:177(prod)
        3    0.000    0.000    0.046    0.015

**Memoria total utilizada**

In [6]:
np.sum(memory_usage((q3_run_v1, (file_path,))))

67014.27734375